In [15]:
import requests #API에 요청과 응답을 받기 위한 라이브러리
import pandas as pd
import json
import math

# --- 설정부 ---

API_KEY = "발급받은 API"
API_ENDPOINT_URL = "https://openapi.gg.go.kr/TBWELFARESSRSM" # API에 요청할 주소 사용
API_NAME_KEY = "TBWELFARESSRSM" # 루프를 종료하기 위해 사용 / API 키 이름
CSV_FILENAME = "gyeonggi_welfare_services_PAJU_test.csv"
TARGET_SIGUN = "파주시" # 타겟하고자하는 곳
PAGE_SIZE = 1000 # API 1회 요청 시 받아올 데이터 최대 개수


print(f"--- 데이터 수집 중 ---")
print(f"  (API: {API_ENDPOINT_URL})")

# --- API 호출부 ---
""" 
    all_rows: 페이지의 데이터 누적용
    current_page: 현재 페이지
    total_count: API에서 알려주는 데이터 갯수 기록용
"""
all_rows = []
current_page = 1
total_count = 0 

base_params = {
    "KEY": API_KEY,
    "Type": "json", # 응답 형식을 JSON으로 요청
    "pSize": PAGE_SIZE, 
}

while True:
    print(f"  > {current_page}페이지 요청 중...")
    params = base_params.copy()
    params["pIndex"] = current_page
    # API에 GET 요청 실행 (10초간 응답 없으면 타임아웃)
    resp = requests.get(API_ENDPOINT_URL, params=params, timeout=10)
    # 응답받은 JSON 문자열을 파이썬 딕셔너리(data)로 변환
    data = resp.json() 
   
    
    # 종료 로직(Rule)
    # 응답에 우리가 기대한 데이터 키(API_NAME_KEY)가 없는 경우, 오류이거나 마지막 페이지로 간주
    if API_NAME_KEY not in data:
        # RESULT 키가 있고 그 안의 'CODE'가 'INFO-300'(데이터 없음)인지 확인
        if "RESULT" in data and data["RESULT"].get("CODE") == "INFO-300":
            print("  ✓ 데이터 없음 (INFO-300). - 종료 -.")
            break
        else:
            raise ValueError(f"API 응답에 '{API_NAME_KEY}' 키가 없습니다. (API 키와 URI 확인 요망)")

    sections = data[API_NAME_KEY]

    # head에서 개수 파악 (첫 페이지 일 경우만 실행)
    if current_page == 1:
        head = sections[0]["head"]
        total_count = int(head[0]["list_total_count"]) 
        
        total_pages = math.ceil(total_count / PAGE_SIZE)
        print(f"API에서 찾은 총 데이터 수: {total_count}개 (약 {total_pages} 페이지)")
        if total_count == 0:
            print("데이터가 0개라 종료합니다.")
            break
    
    # row에서 데이터 추출
    rows = sections[1]["row"]
    all_rows.extend(rows)
    print(f"{len(rows)}개 데이터 추가 (현재 총 {len(all_rows)}개)")

    # 마지막 페이지인지 확인
    if (total_count > 0 and len(all_rows) >= total_count) or (len(rows) < PAGE_SIZE):
        print(" 마지막 페이지, 데이터 수집 완료.")
        break
        
    current_page += 1
    
print(f"\n---데이터 필터링 및 CSV 저장 ---")

df_all = pd.DataFrame(all_rows)
print(f"{len(df_all)}개의 경기도 전체 데이터 수집 완료.")

df_filtered = df_all[df_all["SIGUN_NM"] == TARGET_SIGUN].copy()

print(f" '{TARGET_SIGUN}' 데이터 {len(df_filtered)}개 선별 완료.")

#  --- 컬럼 선택 및 CSV 저장(데이터 필터링) ---
"""
https://data.gg.go.kr/portal/data/service 참고

1	LIST_TOTAL_COUNT:	행총건수
2	CODE:	응답결과코드
3	MESSAGE:	응답결과메세지
4	API_VERSION:	API버전
5	SIGUN_NM:	시군명
6	SERVC_NM:	서비스명
7	SPORT_TARGET:	지원대상
8	SPORT_CYCL:	지원주기
9	CHARGE_DEPT_NM:	담당부서명
10	APLCATN_METH:	신청방법
11	SERVC_RINK_ADDR:	서비스링크주소"
"""
target_columns = [
    "SIGUN_NM",
    "SERVC_NM",
    "SPORT_TARGET",
    "SERVC_RINK_ADDR",
    "CHARGE_DEPT_NM",
    "APLCATN_METH",
    "SPORT_CYCL"
]

df_final = df_filtered[target_columns]
df_final.to_csv(CSV_FILENAME, index=False, encoding="utf-8-sig")

print(f" '{CSV_FILENAME}' 파일 저장 완료. (총 {len(df_final)}개 저장)")
print("\n--- 종료 ---")

--- 데이터 수집 중 ---
  (API: https://openapi.gg.go.kr/TBWELFARESSRSM)
  > 1페이지 요청 중...
API에서 찾은 총 데이터 수: 663개 (약 1 페이지)
663개 데이터 추가 (현재 총 663개)
 마지막 페이지, 데이터 수집 완료.

---데이터 필터링 및 CSV 저장 ---
663개의 경기도 전체 데이터 수집 완료.
 '파주시' 데이터 17개 선별 완료.
 'gyeonggi_welfare_services_PAJU_test.csv' 파일 저장 완료. (총 17개 저장)

--- 종료 ---
